In [15]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
from bs4 import BeautifulSoup
import re
from tqdm.notebook import tqdm
import pandas as pd
import requests

In [16]:
url = 'https://nyheder.tv2.dk/seneste'
driver = webdriver.Chrome()
driver.get(url)

headers = {
    'Name': 'Mathias Udsen', 
    'E-mail':'lcv842@alumni.ku.dk',
    'Message': 'Scraping as a part of our exam project at UCPH, Intoduction to social data science' 
}
requests.get(url, headers)

<Response [200]>

In [17]:
cookie = driver.find_element(By.CSS_SELECTOR, '#onetrust-reject-all-handler')
cookie.click()

In [11]:
Vis_flere = driver.find_element(By.CSS_SELECTOR, '.js-tc_load-more__bound')
Vis_flere.click()

In [33]:
"""
i = 0
while i <= 10:
    try:
        Vis_flere = driver.find_element(By.CSS_SELECTOR, '.js-tc_load-more__bound')
        Vis_flere.click()
        time.sleep(1)
        i = i + 1
    except:
        time.sleep(1)
"""

In [39]:
"""
i = 0
count_of_one_column_lists = 0
while i <= 10:
    try:
        Vis_flere = driver.find_element(By.CSS_SELECTOR, '.js-tc_load-more__trigger.tc_button--secondary.tc_button--default.js-tc_load-more__bound')
        Vis_flere.click()

        new_count_of_one_column_lists = 0
        while new_count_of_one_column_lists <= count_of_one_column_lists:
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            new_count_of_one_column_lists = len(soup.find_all('ul', {'data-columns': '1'}))
            time.sleep(0.25)

        count_of_one_column_lists = new_count_of_one_column_lists
        i += 1
    except:
        time.sleep(1)
"""


KeyboardInterrupt



In [18]:
html = driver.page_source
soup = BeautifulSoup(html, 'lxml')

Headline = []
Category = []
Authors = []
Time = []
Live = []
All_text = []
nummer=0

Live_find1 = False
Live_find2 = False

max_retries = 30
retries = 0

Mathias = 'Ja'

#max_iter = 1001
max_iter = 5
pbar1 = tqdm(total=max_iter, desc="Loading content")
#i = 1000
i = 0
count_of_one_column_lists = 0
if Mathias == 'Ja':
    #while i < 2001:
    while i < 5:
        try:
            Vis_flere = driver.find_element(By.CSS_SELECTOR, '.js-tc_load-more__trigger.tc_button--secondary.tc_button--default.js-tc_load-more__bound')
            Vis_flere.click()

            new_count_of_one_column_lists = 0
            while new_count_of_one_column_lists <= count_of_one_column_lists:
                soup = BeautifulSoup(driver.page_source, 'html.parser')
                new_count_of_one_column_lists = len(soup.find_all('ul', {'data-columns': '1'}))
                time.sleep(0.25)

            count_of_one_column_lists = new_count_of_one_column_lists
            i += 1
            pbar1.update(1)
        except:
            time.sleep(1)
    pbar1.close()

pbar2 = tqdm(total=max_iter, desc="Scraping")
#for i in range(1000, 2001, 1):
for i in range(5):
    retries = 0
    while retries < max_retries:
        try:
            one_column_lists = soup.find_all('ul', {'data-columns': '1'})[i]
            break
        except:
            retries = retries + 1
            time.sleep(1)
    
    if retries == max_retries:
        print(f"Max tries at: Number {nummer} and i: {i}")
    
    for one_column_list in one_column_lists:
        artikler = one_column_list.find_all(class_='tc_teaser')

        for artikel in artikler:
            nummer = nummer + 1
            
            if artikel.find('span', class_='tc_label tc_label--color-live'):
                Live_find1 = True
            else:
                Live_find1 = False
            
            Category.append(artikel.find('span', class_='tc_label tc_label--color-nyheder').text)
            
            href = artikel.find('a', class_='tc_teaser__link').get('href') #if link else None

            text = []
            if href:
                driver.execute_script(f'window.open("{href}", "_blank");')
                driver.switch_to.window(driver.window_handles[-1])

                new_html = driver.page_source
                new_soup = BeautifulSoup(new_html, 'lxml')

                #Category.append(new_soup.find('div', class_='tc_page__label').find('a', class_='tc_label tc_label--color-nyheder').text)
                #Headline.append(new_soup.find('header', class_="tc_page__header").find('h1').text)
                
                liveblog_body = new_soup.find('div', class_="tc_liveblog__body", attrs={'data-adobe-context': "article-body"})
                if liveblog_body:
                    em_tag = liveblog_body.find('em')
                    if em_tag and em_tag.text == 'Denne liveblog opdateres ikke længere.':
                        Live_find2 = True
                    else:
                        Live_find2 = False
                else:
                    Live_find2 = False
                
                if Live_find1 == True or Live_find2 == True:
                    Headline.append(new_soup.find('header', class_="tc_page__header").find('h1').text)
                
                Authors.append(new_soup.find('div', class_="tc_byline").find('strong').text)
                Time.append(new_soup.find('time', class_='tc_timestamp').get('datetime'))
                
                if liveblog_body:
                    article_body_div = liveblog_body
                else:
                    article_body_div = new_soup.find('div', class_='tc_page__body', attrs={'data-adobe-context': 'article-body'})
                
                if article_body_div:
                    rich_content_paragraphs = article_body_div.find_all('p')

                    for paragraph in rich_content_paragraphs:
                        text.append(paragraph.text)

                All_text.append(text)

                driver.close()
                driver.switch_to.window(driver.window_handles[0])
            
            if Live_find1 == True or Live_find2 == True:
                Live.append(1)
            else:
                Live.append(0)
            
            if Live_find1 == False and Live_find2 == False:
                Headline.append(artikel.find('a', class_='tc_teaser__link').get('aria-label'))

    pbar2.update(1)

            #print(nummer, Category[nummer-1], ':', Headline[nummer-1])
    Vis_flere = driver.find_element(By.CSS_SELECTOR, '.js-tc_load-more__bound')
    Vis_flere.click()
    #print(i)
pbar2.close()

Loading content:   0%|          | 0/5 [00:00<?, ?it/s]

Scraping:   0%|          | 0/5 [00:00<?, ?it/s]

Scraping:  80%|████████  | 4/5 [02:10<00:32, 32.71s/it]


In [19]:
print(len(Category), Category[0])
print(len(Authors), Authors[0])
print(len(Headline), Headline[0])
print(len(Time), Time[0])
print(len(Live), Live[0])
print(len(All_text), All_text[0])

129 Lokalt
129  Pernille Leftes
129 Jysk skole opretter 0. klasse med bare tre elever
129 2023-08-14T11:04:48.740Z
129 0
129 ['Mange spændte børn har mandag første skoledag i 0. klasse. Også i Ørslevkloster, hvor 0. klasse i år består af kun tre nye elever.', 'Petrina, Anna og Olivia. ', 'Navneopråbet på de nye elever i 0. klasse på skolen i Ørslevkloster er hurtig overstået, for årgangen består af kun tre piger. Det er dog ikke noget, der bekymrer Annas forældre.', '- Vi er slet ikke utrygge, fordi det personale, der er her, er virkelig gode til børnene. Og fagligheden fejler ikke noget, og de er gode til at se hvert enkelt barn, siger Mette Vestergaard Kristensen. ', 'De tre nye skoleelever skal dog ikke nøjes med hinandens selskab. Deres årgang bliver fra start slået sammen med både 1. og 2. klasse, så de bliver 23 elever på holdet:', '- Så lærer de jo af de ældre børn. Jeg tror faktisk, at det er en god idé i forhold til skolestart, at de er sammen med nogle, der har gået lidt i sk

In [6]:
data = {
    'Category': Category,
    'Authors': Authors,
    'Headline': Headline,
    'Time': Time,
    'Live': Live,
    'All_text': All_text
}

df = pd.DataFrame(data)
df.head(60)

,Category,Authors,Headline,Time,Live,All_text
0,Lokalt,Pernille Leftes,Jysk skole opretter 0. klasse med bare tre elever,2023-08-14T11:04:48.740Z,0,[Mange spændte børn har mandag første skoledag...
1,Samfund,Ritzau,"Ekspert: Rusland sender signal om, at de også ...",2023-08-14T11:01:59.766Z,0,"[Russiske fly over Nordsøen er et signal om, a..."
2,Udland,Ritzau,Desperate pårørende og hunde leder efter overl...,2023-08-14T10:21:08.678Z,0,[På sociale medier beder borgere fra Maui-øen ...
3,Lokalt,Mie Leonora Heiberg,Nordjyde vil hoppe uafbrudt i 13 timer i streg...,2023-08-14T10:07:49.929Z,0,[Ditte Leere Hesselborgs rekordforsøg i Søpark...
4,Krigen i Ukraine,Mikkel Secher,"""Alvorlig hindring"" bremser Ukraines modoffens...",2023-08-14T09:33:59.711Z,0,[Ukraine kæmper med mangel på minerydningsmand...
5,Udland,Ritzau,Australsk kvinde serverede giftig middag: Tre ...,2023-08-14T09:21:49.034Z,0,[I opsigtsvækkende australsk sag afviser kvind...
6,Politik,Rikke Struck Westersø,"Dilemmaer i vente for SF, mener politisk redaktør",2023-08-14T08:44:20.010Z,1,[Hvad skete der? SF holdt sommergruppemøde ved...
7,Lokalt,Marie Kjempff,Dyreejere vågnede op til makabert syn: - Det h...,2023-08-14T08:42:03.887Z,0,[Ubudne gæster i Dragør er stukket af med høns...
8,Lokalt,Iben Sørensen,Skolepatruljer chikaneres i trafikken,2023-08-14T08:34:59.764Z,0,[Mandag morgen var politiet på plads ved Rundh...
9,Politik,Rikke Steenbech,Hård kritik af regeringens klimapolitik: - Det...,2023-08-14T08:28:29.233Z,0,"[I SF mener de ikke, at regeringen er ambitiøs..."


In [41]:
df.to_csv('Data.csv', index=False)

In [8]:
df.to_csv('T:\Data.csv', index=False)